In [ ]:
# Install AutoGen if needed
!pip install pyautogen

In [13]:
#!pip install openai

In [ ]:
from openai import OpenAI
import os

## Set the API key and model name

MODEL= "gpt-4o"

api_key = "Elelemnta_Walmart_APIKey"

client = OpenAI(api_key=api_key)

In [2]:
# Import required classes
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

In [3]:
# Set your OpenAI API key
config = {
    "model": "gpt-4o",
    "api_key": api_key  # Replace with your OpenAI API key
}

You’re specifying the model and authentication to use with OpenAI’s API.

In [4]:
# 👨‍💻 PythonCoder agent
coder = AssistantAgent(
    name="PythonCoder",
    llm_config=config,
    system_message="You are an expert Python programmer. Write and explain clear, commented code."
)

In [5]:
# 🧐 Reviewer agent
reviewer = AssistantAgent(
    name="Reviewer",
    llm_config=config,
    system_message="You are a strict code reviewer. Offer constructive feedback."
)

• Creates another assistant agent named "Reviewer"
• Also powered by GPT-4
• Role: Act as a strict reviewer
• Will analyze code critically
• Provide suggestions or identify bugs
• Ensure best practices, formatting, security, etc.

In [21]:
from autogen import UserProxyAgent

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={
        "use_docker": False },
    human_input_mode="NEVER",  # Simulates user
    max_consecutive_auto_reply=3
)

In [22]:
# GroupChat setup
group_chat = GroupChat(
    agents=[user_proxy, coder, reviewer],
    messages=[],
    max_round=5
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=config
)

In [ ]:
# 🚀 Start the agent discussion
user_proxy.initiate_chat(
    manager,
    message="Write a function to check if a number is prime, and explain it."
)

2. Create a Custom Tool to Save Code

Add a CodeSaverAgent that receives code and writes it to a file.

In [25]:
import os

class CodeSaverAgent(AssistantAgent):
    def save_code_to_file(self, code: str, filename="generated_code.py"):
        with open(filename, "w") as f:
            f.write(code)
        return f"Code saved to {filename}"

# Inherit and customize response
class CodeSaverAssistant(CodeSaverAgent):
    def _process_message(self, message, sender, config):
        if "save this code" in message.lower():
            code_block = message.split("```python")[1].split("```")[0]
            return self.save_code_to_file(code_block)
        return super()._process_message(message, sender, config)

🔗 3. Integrate into GroupChat

Add this new agent to your group chat:

In [26]:
code_saver = CodeSaverAssistant(
    name="CodeSaver",
    llm_config=config,
    system_message="You save valid Python code sent to you as .py files."
)

group_chat = GroupChat(
    agents=[user_proxy, coder, reviewer, code_saver],
    messages=[],
    max_round=6
)

manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=config
)

🚀 4. Run a Complete Workflow

Update the initial task:

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="Write a prime number checker in Python, explain it, and ask CodeSaver to save it."
)